'''
<br><br>
@Author: Shivraj Yelave<br>
@Date: 12-09-24<br>
@Last modified by: Shivraj Yelave<br>
@Last modified time: 12-09-24<br>
@Title: CURD Operation in lambda Using boto3<br><br>
'''

## Lambda Code

In [ ]:
import json
import boto3

# Initialize S3 client
s3 = boto3.client('s3')

def lambda_handler(event, context):
    
    """
    Description:
    AWS Lambda function entry point.
    
    Parameters:
    - event (dict): The input to the Lambda function. Should contain the operation type and relevant details.
    - context (object): Contains runtime information for the Lambda function. Not used in this code.
    
    Returns:
    - dict: The HTTP response with status code and message indicating the result of the operation.
    """
    operation = event.get('operation', '').lower()
    
    if operation == 'create':
        return create_file(event)
    elif operation == 'read':
        return read_file(event)
    elif operation == 'update':
        return update_file(event)
    elif operation == 'delete':
        return delete_file(event)
    else:
        return {
            'statusCode': 400,
            'body': json.dumps('Invalid operation. Use "create", "read", "update", or "delete".')
        }

def create_file(event):
    """
    Description:
    Creates a new file in S3.

    Parameters:
    - event (dict): Contains 'bucket_name', 'file_name', and optionally 'file_content'.
      - 'bucket_name' (str): The name of the S3 bucket.
      - 'file_name' (str): The name of the file to create.
      - 'file_content' (str, optional): The content to upload. Defaults to 'Default file content' if not provided.
    
    Returns:
    - dict: The HTTP response with status code and message indicating the result of the creation operation.
    """
    bucket_name = event['bucket_name']
    file_name = event['file_name']
    file_content = event.get('file_content', 'Default file content')
    
    # Upload the file to S3
    s3.put_object(Bucket=bucket_name, Key=file_name, Body=file_content)
    
    return {
        'statusCode': 201,
        'body': json.dumps(f"File {file_name} created successfully in {bucket_name}.")
    }

def read_file(event):
    """
    Description:
    Reads an existing file from S3.

    Parameters:
    - event (dict): Contains 'bucket_name' and 'file_name'.
      - 'bucket_name' (str): The name of the S3 bucket.
      - 'file_name' (str): The name of the file to read.
    
    Returns:
    - dict: The HTTP response with status code and file content if found. Returns a 404 error if the file is not found.
    """
    bucket_name = event['bucket_name']
    file_name = event['file_name']
    
    try:
        # Download file content
        response = s3.get_object(Bucket=bucket_name, Key=file_name)
        file_content = response['Body'].read().decode('utf-8')
        
        return {
            'statusCode': 200,
            'body': json.dumps({
                'file_name': file_name,
                'file_content': file_content
            })
        }
    except s3.exceptions.NoSuchKey:
        return {
            'statusCode': 404,
            'body': json.dumps(f"File {file_name} not found in {bucket_name}.")
        }

def update_file(event):
    """
    Description:
    Updates an existing file in S3 by overwriting its content.

    Parameters:
    - event (dict): Contains 'bucket_name', 'file_name', and 'file_content'.
      - 'bucket_name' (str): The name of the S3 bucket.
      - 'file_name' (str): The name of the file to update.
      - 'file_content' (str): The new content to upload.
    
    Returns:
    - dict: The HTTP response with status code and message indicating the result of the update operation.
    """
    bucket_name = event['bucket_name']
    file_name = event['file_name']
    new_content = event['file_content']
    
    # Update the file (overwrite its content)
    s3.put_object(Bucket=bucket_name, Key=file_name, Body=new_content)
    
    return {
        'statusCode': 200,
        'body': json.dumps(f"File {file_name} updated successfully in {bucket_name}.")
    }

def delete_file(event):
    """
    Description:
    Deletes a file from S3.

    Parameters:
    - event (dict): Contains 'bucket_name' and 'file_name'.
      - 'bucket_name' (str): The name of the S3 bucket.
      - 'file_name' (str): The name of the file to delete.
    
    Returns:
    - dict: The HTTP response with status code and message indicating the result of the deletion operation. Returns a 404 error if the file is not found.
    """
    bucket_name = event['bucket_name']
    file_name = event['file_name']
    
    try:
        # Delete the file from S3
        s3.delete_object(Bucket=bucket_name, Key=file_name)
        
        return {
            'statusCode': 200,
            'body': json.dumps(f"File {file_name} deleted successfully from {bucket_name}.")
        }
    except s3.exceptions.NoSuchKey:
        return {
            'statusCode': 404,
            'body': json.dumps(f"File {file_name} not found in {bucket_name}.")
        }
